In [1]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import os
import dimod
from dwave.system import LeapHybridBQMSampler #LeapHybridSampler

In [2]:
# --- Generate couplings ---
def generate_J(N, seed=None):
    # Create directory if it doesn't exist
    os.makedirs('generated_data', exist_ok=True)
    
    if seed is not None:
        np.random.seed(seed)
    std = 1.0 / np.sqrt(N)
    data = [(i, j, np.random.normal(0, std)) for i in range(N) for j in range(i+1, N)]
    df = pd.DataFrame(data, columns=['i','j','Jij'])
    
    if seed is not None:
        filepath = f'generated_data/Jij_N{N}_seed{seed}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")
    else:
        filepath = f'generated_data/Jij_N{N}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")

# --- Load couplings ---
def load_J(N, seed=None):
    filename = f'generated_data/Jij_N{N}_seed{seed}.csv' if seed is not None else f'generated_data/Jij_N{N}.csv'
    df = pd.read_csv(filename)
    N = max(df['i'].max(), df['j'].max()) + 1
    return [(int(i), int(j), float(Jij)) for i,j,Jij in df.values], N

In [3]:
# --- Build and solve D-Wave BQM model ---
def solve_SK_dwave(rows, N, seed=None, time_limit=30):
    # Create directory if it doesn't exist
    os.makedirs('dwave_solutions', exist_ok=True)
    # Build Binary Quadratic Model (BQM) in SPIN variables
    h = {i: 0.0 for i in range(N)} # no onsite field
    J = {(i, j): -Jij for i, j, Jij in rows} # match SK Hamiltonian: H = -sum Jij S_i S_j


    bqm = dimod.BinaryQuadraticModel(h, J, dimod.SPIN)


    sampler = LeapHybridBQMSampler() #LeapHybridSampler()
    sampleset = sampler.sample(bqm, time_limit=time_limit)


    result = sampleset.first.sample
    S = [result[i] for i in range(N)]
    energy = min(sampleset.record.energy)


    print(f"N = {N}\nEnergy = {energy}") 
    print(f"First spins: {S}")

    filepath = f'dwave_solutions/solution_N{N}_seed{seed}.csv'

    pd.DataFrame({'i': range(N), 'Energy_dwave': energy, 'S': S}).to_csv(filepath, index=False)

    print(f"Saved solution to {filepath}")
    return energy, S

In [4]:
N = 500  # choose system size
energy_dwave_list = []
S_dwave_list = []
for i in range(1, 41):
    generate_J(N, seed=i*5)
    rows, N = load_J(N, seed=i*5)
    energy_dwave, S_dwave = solve_SK_dwave(rows, N, seed=i*5, time_limit=45) 
    energy_dwave_list.append(energy_dwave)
    S_dwave_list.append(S_dwave)

Saved couplings for N=500 to generated_data/Jij_N500_seed5.csv
N = 500
Energy = -377.42250492881703
First spins: [-1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, 1, -1, -1, 1, -1, -1, 1, -1, 1, 1, 

In [6]:
energy_dwave_list = []
for l in range(1,41):
    df_result_dwave = pd.read_csv(f"dwave_solutions/solution_N500_seed{l*5}.csv")
    energy_dwave_list.append(df_result_dwave['Energy_dwave'][0])

In [7]:
energy_dwave_array = np.array(energy_dwave_list)
energy_dwave_mean = np.mean(energy_dwave_array)
energy_dwave_std = np.std(energy_dwave_array)
print(f"Average D-Wave Energy over 40 instances: {energy_dwave_mean} ± {energy_dwave_std}")

print("mean energy per site:", energy_dwave_mean / N)

Average D-Wave Energy over 40 instances: -375.5608498082692 ± 3.038233983024407
mean energy per site: -0.7511216996165384


In [19]:
N=500  # number of sites
energy_dwave_list = []
for l in range(1,41):
    df_result_dwave = pd.read_csv(f"dwave_solutions/solution_N500_seed{l*5}.csv")
    energy_dwave_list.append(df_result_dwave['Energy_dwave'][0])

# Get the 40 lowest energy configurations
num_lowest = 40
sorted_indices = np.argsort(energy_dwave_list)*5
lowest_40_seeds = sorted_indices[:num_lowest]
lowest_40_energies = [energy_dwave_list[int(idx/5)] for idx in lowest_40_seeds]

print(f"Found {num_lowest} lowest energy configurations:")
print(f"\nTop 40 seeds with lowest energies:")
for i in range(40):
    seed_idx = lowest_40_seeds[i]
    energy = lowest_40_energies[i]
    print(f"  Rank {i+1}: Seed {seed_idx}, Energy = {energy:.6f}")

print(f"\nEnergy range of 40 lowest:")
print(f"  Min (seed {lowest_40_seeds[0]}): {lowest_40_energies[0]:.6f}")
print(f"  Max (seed {lowest_40_seeds[-1]}): {lowest_40_energies[-1]:.6f}")
print(f"  Mean: {np.mean(lowest_40_energies):.6f}")
print(f"  Std:  {np.std(lowest_40_energies):.6f}")

# Store for later use
min_energy_idx = lowest_40_seeds[0]
min_energy = lowest_40_energies[0]

Found 40 lowest energy configurations:

Top 40 seeds with lowest energies:
  Rank 1: Seed 115, Energy = -384.173225
  Rank 2: Seed 25, Energy = -384.099947
  Rank 3: Seed 145, Energy = -382.972401
  Rank 4: Seed 75, Energy = -379.393313
  Rank 5: Seed 120, Energy = -378.952000
  Rank 6: Seed 165, Energy = -378.714759
  Rank 7: Seed 195, Energy = -377.807208
  Rank 8: Seed 0, Energy = -377.422505
  Rank 9: Seed 15, Energy = -376.854207
  Rank 10: Seed 30, Energy = -376.605014
  Rank 11: Seed 175, Energy = -376.401370
  Rank 12: Seed 135, Energy = -376.300336
  Rank 13: Seed 125, Energy = -376.103380
  Rank 14: Seed 10, Energy = -375.897248
  Rank 15: Seed 155, Energy = -375.875158
  Rank 16: Seed 50, Energy = -375.680386
  Rank 17: Seed 60, Energy = -375.644835
  Rank 18: Seed 5, Energy = -375.310173
  Rank 19: Seed 70, Energy = -375.274657
  Rank 20: Seed 150, Energy = -375.193207
  Rank 21: Seed 170, Energy = -374.918476
  Rank 22: Seed 185, Energy = -374.879191
  Rank 23: Seed 100, E

In [18]:
# energy_dwave_list